<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#AB-Testing" data-toc-modified-id="AB-Testing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>AB-Testing</a></span></li></ul></div>

# AB-Testing

![cats](images/cats.jpeg)


Imaginad que somos los cientificos de datos de la empresa de videojuegos Tactile Entertainment. Los desarrolladores del juego Cookie Cats pretenden introducir un cambio en el juego para aumentar la retencion de los jugadores. En cierto nivel del juego los jugadores se encuentran una puerta que les obliga a esperar o a pagar la app. Actualmente la puerta se encuentra en nivel 30 y se pretende pasar al nivel 40, para comprobar la retencion a 1 y 7 dias. Antes de realizar el cambio definitivo en el juego se raliza un test AB.

Los datos estan alojados en `data/cookie_cats.csv`. Nuestro grupo de control sera la version actual `gate_30` y el grupo de tratamiento sera la version `gate_40`. Debemos realizar el test para 1 dia de retencion `retention_1` y para 7 dias `retention_7`.

In [119]:
# librerias

import pandas as pd
import numpy as np
import statsmodels.stats.api as sms
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from scipy.stats import norm, sem

import seaborn as sns

In [154]:
efecto=sms.proportion_effectsize(0.13, 0.15)  # calculo el efecto desde el 13% media actual al 15% media objetivo

efecto

-0.0576728617308947

In [155]:
n_requerido=sms.NormalIndPower().solve_power(efecto, power=0.8, alpha=0.05)
n_requerido

4719.4740575998185

In [4]:
cookie= pd.read_csv(r'C:\Users\Héctor\Documents\IRONHACK\DATA ANALIST\BOOTCAMP\Labs\Semana 6\6.4_lab-AB-Testing\data\cookie_cats.csv')
cookie

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
...,...,...,...,...,...
90184,9999441,gate_40,97,True,False
90185,9999479,gate_40,30,False,False
90186,9999710,gate_30,28,True,False
90187,9999768,gate_40,51,True,False


In [171]:
# transformacion
cookie.version.value_counts()

gate_40    45489
gate_30    44700
Name: version, dtype: int64

In [173]:
# muestreo
control = cookie[cookie.version =='gate_30'].sample(n=44700, random_state=42) 

tratamiento = cookie[cookie.version=='gate_40'].sample(n=44700, random_state=42)

ab_test = pd.concat([control, tratamiento], axis=0)

ab_test.reset_index(drop=True, inplace=True)

ab_test.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,6531033,gate_30,35,False,False
1,8003009,gate_30,4,False,False
2,5566807,gate_30,23,True,False
3,64235,gate_30,1,False,False
4,6210551,gate_30,7,False,False


In [174]:
ab_test.tail()

,userid,version,sum_gamerounds,retention_1,retention_7
89395,8755807,gate_40,26,True,False
89396,8288070,gate_40,80,True,False
89397,9469122,gate_40,1,False,False
89398,1056682,gate_40,106,True,False
89399,1012652,gate_40,199,True,True


In [175]:
ab_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89400 entries, 0 to 89399
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          89400 non-null  int64 
 1   version         89400 non-null  object
 2   sum_gamerounds  89400 non-null  int64 
 3   retention_1     89400 non-null  bool  
 4   retention_7     89400 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [193]:
control_res_30_1=ab_test[['retention_1']] [ab_test.version=='gate_30']

change_res_40_1= ab_test[['retention_1']] [ab_test.version=='gate_40']

In [201]:
control_res_30_1.count()

retention_1    44700
dtype: int64

In [208]:
# testeo para retention_1
observaciones = [44700]  

conversiones=[control_res_30_1.sum(), change_res_40_1.sum()]  # conversiones por grupo
z_score, p_valor = proportions_ztest(conversiones, nobs=observaciones)


# intervalos de confianza

(control_30_1_a, tratamiento_40_1_a), (control_30_1_b, tratamiento_40_1_b) = proportion_confint(conversiones, 
                                                                            nobs=observaciones,
                                                                            alpha=0.05)

In [218]:
print (' el p valor es', p_valor, 'el z_score es', z_score)
print('el intervaleo de control es', control_30_1_a, control_30_1_b)
print('el intervalo de cambio es', tratamiento_40_1_a, tratamiento_40_1_b)


 el p valor es [0.09121036] el z_score es [1.68904731]
el intervaleo de control es [0.44357772] [0.45279812]
el intervalo de cambio es [0.43796823] [0.44717719]


In [203]:
control_res_30_7=ab_test[['retention_7']] [ab_test.version=='gate_30']

change_res_40_7= ab_test[['retention_7']] [ab_test.version=='gate_40']



In [222]:
# testeo para retention_7
observaciones = [44700]  #control_res.count()

conversiones=[control_res_30_7.sum(), change_res_40_7.sum()]  # conversiones por grupo
z_score, p_valor = proportions_ztest(conversiones, nobs=observaciones)


# intervalos de confianza

(control_30_7_a, tratamiento_40_7_a), (control_30_7_b, tratamiento_40_7_b) = proportion_confint(conversiones, 
                                                                            nobs=observaciones,
                                                                            alpha=0.05)


In [223]:
print (' el p valor es', p_valor, 'el z_score es', z_score)
print('el intervaleo de control es', control_30_7_a, control_30_7_b)
print('el intervalo de cambio es', tratamiento_40_7_a, tratamiento_40_7_b)

 el p valor es [0.00287681] el z_score es [2.98060371]
el intervaleo de control es [0.18656312] [0.19383957]
el intervalo de cambio es [0.17885823] [0.18601872]


In [ ]:
# conclusiones
# Explica detalladamente las conclusiones obtenidas de. tu testeo.